# GeoSpatial Analysis

NOTE: TUTORIAL UNDER CONSTRUCTION.

<div class="alert alert-success">
...
</div>

<div class="alert alert-info">
<a href=https://en.wikipedia.org/wiki/Statistical_classification>Classification</a>
</div>

## Using the Google Maps API

In [ ]:
# Import
%matplotlib inline

#import folium
import requests

In [ ]:
# Set the Google Maps API URL
url = 'https://maps.googleapis.com/maps/api/geocode/json'

# Set the parameters to use with the URL
#  Here we set an 'address' field with the address we wish to search for
params = {'address': 'UCSD Peterson Hall, Peterson Hall, San Diego, CA'}

In [ ]:
# Make the URL request
req_result = requests.get(url, params=params)

In [ ]:
# Pull out the results field of the Response object
results = req_result.json()['results']

In [ ]:
# Grab the location data
#location = results[0]['geometry']['location']

# Check the result - lat, long data
#print(location['lat'], location['lng'])

In [ ]:
type(req_result)

In [ ]:
results

# Python packages for playing with geospatial data

* haversine: computes straight line distance
* folum: python wrapper for leaflet.js

In [ ]:
%load_ext autoreload
%matplotlib inline
%config InlineBackend.figure_format = 'retina' # high res plotting

import folium
# pip install folium
# https://folium.readthedocs.io/en/latest

from haversine import haversine, Unit
# pip install haversine
# https://pypi.python.org/pypi/haversine

import geocoder
# pip install geocoder
# https://pypi.python.org/pypi/geocoder

import shapefile
# pip install pyshp
# shapefile package: pyshp
# https://pypi.python.org/pypi/pyshp

In [ ]:
import numpy as np
import requests

import seaborn as sns
sns.set_style('whitegrid')
sns.set_style("whitegrid", {'axes.grid' : False})

import matplotlib
import matplotlib.pyplot as plt

# plot settings
font = {'family' : 'Bitstream Vera Sans',
        'weight' : 'regular',
        'size'   : 13}
figure = {'figsize' : (10,8)}

matplotlib.rc('font', **font)
matplotlib.rc('figure', **figure)

In [ ]:
# simplified method using geocoder package
# NOTE that I *highly* recommend you get an API key from
    # google and use geocoder.google instead of geocoder.geocodefarm

g = geocoder.geocodefarm('Price Center, La Jolla, CA')
g.json


In [ ]:
print(g.latlng)

In [ ]:
# reverse geocoding a lat/lng to an address
g = geocoder.geocodefarm(g.latlng, method='reverse')
g.json


In [ ]:
# working with shapefiles
# neighborhood shapefile data from Zillow:
# https://www.zillow.com/howto/api/neighborhood-boundaries.htm

sf = shapefile.Reader("ZillowNeighborhoods-RI.shp")
shapes = sf.shapes()


In [ ]:
 shapes[0].points

In [ ]:
j=0
for i in range(len(shapes[j].points)):
    plt.scatter(shapes[j].points[i][0], shapes[j].points[i][1], c='k')

plt.show()

## Working with shapefiles
#### There are many issues with geospatial analyses, as I outlined in lecture.
#### One is how you aggregate data so that you can analyze them.
#### One way is to break up a geographic region into equally-size units (hexagonal tiling, for example).
#### But, for the sake of interpretability and intuitiveness, neighborhood boundaries work quite well.
#### Here's one way of doing that using shapefiles, which are just a way of storing geographic boundaries as sets of lat/lng pairs that outline a region and associate it with metadata about that region.
#### Thankfully the real estate website Zillow has done this hard work and made the shapefiles freely available.

In [ ]:
# read the in the shapefile and list the methods associated with the object
sf = shapefile.Reader("ZillowNeighborhoods-CA.shp")
dir(sf)

In [ ]:
# read in the dbf (metadata) file and list all the methods associated with it
sfdbf = shapefile.Reader("ZillowNeighborhoods-CA.dbf")
dir(sfdbf)

In [ ]:
metadata = sfdbf.shapeRecords()
metadata[38].record


In [ ]:
# find indices of all San Diego neighborhoods
sd_list = []
counter = 0

for i in range(len(metadata)):
    if metadata[i].record[2] == 'San Diego':
        sd_list.append(i)
        counter += 1


In [ ]:
shapes = sf.shapes()

sd_shapes = []

for i in range(len(sd_list)):
    sd_shapes.append(shapes[sd_list[i]].points)

for i in range(len(sd_shapes)):
    for j in range(len(sd_shapes[i])):
        sd_shapes[i][j] = sd_shapes[i][j][::-1]


In [ ]:
temp_shapes = sd_shapes[0:5]

for i in range(len(temp_shapes)):
    for j in range(len(temp_shapes[i])):
        plt.scatter(temp_shapes[i][j][0], temp_shapes[i][j][1], s=1, c='k')

plt.show()


In [ ]:
# two coordinates

coordinate_1 = [32.8811, -117.2375]
coordinate_2 = [32.8709, -117.2108]


In [ ]:
# Computes the distance between two points in kilometers
ditance_in_km = haversine(coordinate_1, coordinate_2)
ditance_in_km

In [ ]:
# Distance in miles
ditance_in_mi = haversine(coordinate_1, coordinate_2, unit=Unit.MILES)
ditance_in_mi

In [ ]:
lat = coordinate_1[0]
lon = coordinate_1[1]
zoom_start = 14

m = folium.Map(location=[lat, lon], zoom_start=zoom_start)

kw = dict(fill_color='red', radius=20)
c0 = folium.CircleMarker(coordinate_1, **kw)
c1 = folium.CircleMarker(coordinate_2, **kw)

for c in [c0, c1]:
    m.add_child(c)

m

In [ ]:
zoom_start = 10
m = folium.Map(location=[lat, lon], zoom_start=zoom_start, tiles='Stamen Toner')

for c in range(len(sd_shapes)):
    hood_line = folium.PolyLine(locations=sd_shapes[c], weight=2, color = 'blue')
    m.add_child(hood_line)

m


In [ ]:
# GET COLORS HERE: http://colorbrewer2.org

zoom_start = 10
m = folium.Map(location=[lat, lon], zoom_start=zoom_start, tiles='Stamen Toner')

for c in range(len(sd_shapes)):
    hood_line = folium.features.PolyLine(locations=sd_shapes[c], color='#FF0000', fill_color='#fc8d50', weight=5)
    m.add_child(hood_line)

m